In [1]:
!pip install delta-spark==1.0

In [2]:
import pyspark
from delta import *
from pyspark.sql import SparkSession

bucket = "e-drill"

builder = pyspark.sql.SparkSession.builder.appName("jupyter-pyspark") \
        .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.1.2,org.apache.spark:spark-avro_2.12:3.1.2")\
        .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
        .config("spark.hadoop.fs.s3a.access.key", "minio") \
        .config("spark.hadoop.fs.s3a.secret.key", "SU2orange!") \
        .config("spark.hadoop.fs.s3a.fast.upload", True) \
        .config("spark.hadoop.fs.s3a.path.style.access", True) \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \

spark = configure_spark_with_delta_pip(builder).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f3e40b78-5bf2-459f-95e5-2ca63ca6f24c;1.0
	confs: [default]


:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 189ms :: artifacts dl 4ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-runtime;3.5.2 from central in [default]
	org.antlr#antlr4;4.7 from central in [default]
	org.antlr#antlr4-runtime;4.7 from central in [default]
	org.glassfish#javax.json;1.0.4 from central in [default]
	---------------------------------------------------------------------
	|             

In [3]:
data = spark.range(0, 5)
data.write.format("delta").save("/tmp/delta-table")

AnalysisException: file:/tmp/delta-table already exists.

In [4]:
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()

+---+
| id|
+---+
|  7|
|  6|
|  8|
|  9|
|  5|
+---+



In [8]:
data = spark.range(5, 10)
data.write.format("delta").mode("overwrite").save("/tmp/delta-table")

In [9]:
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()

+---+
| id|
+---+
|  7|
|  6|
|  8|
|  9|
|  5|
+---+



In [11]:
df = spark.read.format("delta").option("versionAsOf", 1).load("/tmp/delta-table")
df.show()

+---+
| id|
+---+
|  7|
|  6|
|  8|
|  9|
|  5|
+---+



In [12]:
history = spark.sql("DESCRIBE HISTORY delta.`/tmp/delta-table`")
latest_version = history.selectExpr("max(version)").collect()
df = spark.read.format("delta").option("versionAsOf", latest_version[0][0]).load("/tmp/delta-table")

ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8

In [13]:
df.show()

+---+
| id|
+---+
|  7|
|  6|
|  8|
|  9|
|  5|
+---+



In [15]:
history.show()

+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|      1|2022-12-07 22:49:...|  null|    null|    WRITE|{mode -> Overwrit...|null|    null|     null|          0|          null|        false|{numFiles -> 6, n...|        null|
|      0|2022-12-07 22:49:...|  null|    null|    WRITE|{mode -> ErrorIfE...|null|    null|     null|       null|          null|         true|{numFiles -> 6, n...|        null|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--

In [10]:
stocks = spark.read.csv(f"s3a://{bucket}/stocks.csv",inferSchema=True,header=True) # Read From S3
stocks.write.format("delta").mode("overwrite").save(f"s3a://{bucket}/stocks-table") # Write to DeltaTable on D3

In [11]:
stocksTable = DeltaTable.forPath(spark,f"s3a://{bucket}/stocks-table") # Get as Table

In [15]:
stocksTable.toDF().printSchema()

root
 |-- price: double (nullable = true)
 |-- symbol: string (nullable = true)



In [23]:
stocksTable.update(condition="symbol='AAPL'", set= { 'price' : "409" }) #Update will re-write for you

In [21]:
stocksTable.toDF().write.format("delta").mode("overwrite").save(f"s3a://{bucket}/stocks-table")

In [24]:
history = spark.sql(f"DESCRIBE HISTORY delta.`s3a://{bucket}/stocks-table`")
history.show()

+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|version|          timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|
+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|      3|2022-12-08 00:02:51|  null|    null|   UPDATE|{predicate -> (sy...|null|    null|     null|          2|          null|        false|{numRemovedFiles ...|        null|
|      2|2022-12-08 00:01:05|  null|    null|    WRITE|{mode -> Overwrit...|null|    null|     null|          1|          null|        false|{numFiles -> 1, n...|        null|
|      1|2022-12-07 23:59:36|  null|    null|   UPDATE|{predicate -> (sy...|null|    null|     null|          0|        